In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from models.neural_network import neural_network
from utils.datasets import load_cats
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
cats = os.getenv('CATS')
train_file = os.path.join(cats, 'train.h5')
test_file = os.path.join(cats, 'test.h5')
x_train, y_train, x_test, y_test, classes = load_cats(train_file, test_file)

In [18]:
def keras_model(input_shape, name='model'):
    """
    Keras model
    
    Arguments:
    input_shape -- shape of the images of the dataset
        (height, width, channels) as a tuple.  
        Note that this does not include the 'batch' as a dimension.
        If you have a batch like 'X_train', 
        then you can provide the input_shape using
        X_train.shape[1:]
    """    

    X_input = keras.Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = layers.ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = layers.Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = layers.BatchNormalization(axis = 3, name = 'bn0')(X)
    X = layers.Activation('relu')(X)

    # MAXPOOL
    X = layers.MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = layers.Flatten()(X)
    X = layers.Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = keras.Model(inputs = X_input, outputs = X, name=name)
    
    return model

In [35]:
y_train_reshaped = y_train.reshape(209,1) # from (1,209)
y_test = y_test.reshape(50,1) 

In [19]:
model = keras_model(x_train.shape[1:])

In [20]:
model.compile(optimizer = "adam", loss = 'binary_crossentropy', metrics = ["accuracy"])

In [30]:
model.fit(x = x_train, y = y_train_reshaped, epochs = 20, batch_size = 64)

Train on 209 samples
Epoch 1/20
209/209 [==============================] - 0s 1ms/sample - loss: 0.0056 - accuracy: 1.0000
Epoch 2/20
209/209 [==============================] - 0s 325us/sample - loss: 0.0066 - accuracy: 1.0000
Epoch 3/20
209/209 [==============================] - 0s 292us/sample - loss: 0.0060 - accuracy: 1.0000
Epoch 4/20
209/209 [==============================] - 0s 296us/sample - loss: 0.0049 - accuracy: 1.0000
Epoch 5/20
209/209 [==============================] - 0s 297us/sample - loss: 0.0061 - accuracy: 1.0000
Epoch 6/20
209/209 [==============================] - 0s 306us/sample - loss: 0.0069 - accuracy: 1.0000
Epoch 7/20
209/209 [==============================] - 0s 301us/sample - loss: 0.0087 - accuracy: 1.0000
Epoch 8/20
209/209 [==============================] - 0s 298us/sample - loss: 0.0065 - accuracy: 1.0000
Epoch 9/20
209/209 [==============================] - 0s 301us/sample - loss: 0.0053 - accuracy: 1.0000
Epoch 10/20
209/209 [========================

In [37]:
preds = model.predict(x_test)

In [39]:
predictions = (preds > 0.5).astype(float)

In [40]:
test_accuracy = 100 - np.mean(np.abs(predictions - y_test)) * 100

In [41]:
test_accuracy

76.0